<a href="https://colab.research.google.com/github/bt86548/pytorch/blob/master/pytorch5%266%267.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os     
import zipfile

local_zip = '/content/drive/My Drive/kagglecatsanddogs_3367a.zip' #需要解壓縮的檔案
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/PetImages2') #解壓縮的路徑
zip_ref.close()

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
REBUILD_DATA = True

In [ ]:
class DogsVSCats():
    IMG_SIZE = 100 #把所有圖片尺寸都統一,可能會失真需要先測試一下圖片
    CATS = "/content/PetImages2/PetImages/Cat" 
    DOGS = "/content/PetImages2/PetImages/Dog"
    TESTING = "PetImages/Testing"
    LABELS = {CATS: 0, DOGS: 1} #貼標籤
    training_data = []

    
    catcount = 0 #計數很重要 需要知道做完沒
    dogcount = 0


    def make_training_data(self):
        for label in self.LABELS: #用標籤去跑所有的圖像
            for f in tqdm(os.listdir(label)): #跑遍所有圖片,tqdm進度條
                if "jpg" in f:
                    try:
                        path = os.path.join(label, f) #label為/content/PetImages2/PetImages/Cat位置,f為每張照片,以此類推去讀取
                        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE) #把讀到的照片變成灰階
                        img = cv2.resize(img, (self.IMG_SIZE, self.IMG_SIZE)) #把每張照片都變50*50的大小
                        self.training_data.append([np.array(img), np.eye(2)[self.LABELS[label]]])  
                        #np.array(img)為該圖片的矩陣,np.eye(2)[self.LABELS[label]]]
                        # do something like print(np.eye(2)[1]), just makes one_hot  np.eye(2)為對角線為1的2X2矩陣
                        #print(np.eye(2)[self.LABELS[label]]) 

                        if label == self.CATS:
                            self.catcount += 1
                        elif label == self.DOGS:
                            self.dogcount += 1
                    except Exception as e:
                        pass
                        #print(label, f, str(e))

        np.random.shuffle(self.training_data) #重新洗牌讓他更準
        np.save("training_data.npy", self.training_data) #存成npy檔讓圖片比較好辨識
        print('Cats:',dogsvcats.catcount) #計算貓照片的數量
        print('Dogs:',dogsvcats.dogcount) #計算狗照片的數量
if REBUILD_DATA:
    dogsvcats = DogsVSCats()
    dogsvcats.make_training_data()


100%|██████████| 12501/12501 [00:16<00:00, 768.14it/s]


Cats: 12476
Dogs: 12470


In [ ]:
training_data = np.load("/content/training_data.npy", allow_pickle=True) #讀取npy檔
print(len(training_data))

24946


In [ ]:
import torch

X = torch.Tensor([i[0] for i in training_data]).view(-1,100,100)
print(len(X))
print(X)
X = X/255.0
y = torch.Tensor([i[1] for i in training_data])

24946
tensor([[[136., 132., 132.,  ..., 167., 166., 165.],
         [136., 134., 132.,  ..., 171., 175., 170.],
         [137., 135., 133.,  ..., 175., 171., 173.],
         ...,
         [ 70.,  74.,  85.,  ...,  55., 101., 101.],
         [ 84.,  84.,  91.,  ...,  79.,  59.,  82.],
         [ 62.,  64.,  66.,  ...,  65.,  40.,  44.]],

        [[157., 137., 105.,  ..., 245., 153., 128.],
         [152., 132., 107.,  ..., 254., 147., 128.],
         [152., 137., 103.,  ..., 249., 146., 120.],
         ...,
         [ 56.,  64.,  66.,  ...,  49.,  63.,  61.],
         [ 57.,  55.,  64.,  ...,  54.,  55.,  55.],
         [ 56.,  56.,  59.,  ...,  60.,  50.,  54.]],

        [[119., 118., 119.,  ..., 198., 200., 203.],
         [122., 121., 123.,  ..., 198., 198., 198.],
         [128., 126., 126.,  ..., 192., 191., 197.],
         ...,
         [ 76.,  80.,  89.,  ...,  92.,  90.,  89.],
         [ 79.,  81.,  89.,  ...,  91.,  87.,  86.],
         [ 76.,  78.,  90.,  ...,  88.,  84.,  

In [ ]:
# import matplotlib.pyplot as plt

# plt.imshow(X[0], cmap="gray")

In [ ]:
# print(y[0])

In [ ]:
import torch #建立模型
import torch.nn as nn
import torch.nn.functional as F


In [ ]:
class Net(nn.Module):
    def __init__(self):
        #卷積層
        super().__init__() # just run the init of parent class (nn.Module)
        self.conv1 = nn.Conv2d(1, 32,   5) # input is 1 image, 32 output channels, 5x5 kernel / window
        self.conv2 = nn.Conv2d(32, 64,  5) # input is 32, bc the first layer output 32. Then we say the output will be 64 channels, 5x5 conv
        self.conv3 = nn.Conv2d(64, 128, 5) #padding預設為0
        #output_shape = (image_shape-filter_shape+2*padding)/stride + 1
        

        #扁平層:把多維的輸入壓扁為一維輸出，常用在從卷積層到全連接層的過渡，無參數'''
        #flattening.
        x = torch.randn(100,100).view(-1,1,100,100)
        self._to_linear = None
        self.convs(x)

        #全連接層
        self.fc1 = nn.Linear(self._to_linear, 512) 
        self.fc2 = nn.Linear(512, 2) # 512 in, 2 out bc we're doing 2 classes (dog vs cat).
        # self.fc1 = nn.Linear(self._to_linear, 512) 

    def convs(self, x): #三層池化層
        # max pooling over 2x2
        #取最大值(Max)：包括一維(MaxPooling1D)、二維(MaxPooling2D)、三維(MaxPooling3D)，分別處理時序資料、2D 圖形及每一點含其他資訊的2D 圖形。
        #池化層:池化目的只是在將圖片資料量減少並保留重要資訊的方法，把原本的資料做一個最大化或是平均化的降維計算。
        #池化法除了最大化池化法外，也可以做平均池化法(取最大部份改成取平均)、最小化池化法(取最大部份改成取最小化)等。
        print("initial",x.shape)
        print()
        x = self.conv1(x)
        print("conv1", x.shape)
        print()
        x = F.relu(x)
        print("relu1",x.shape)
        print()
        x = F.max_pool2d(x, (2,2)) #max_pool取最大值
        print("max_pool2d 1",x.shape)

        print()
        x = self.conv2(x)
        print("conv2", x.shape)
        print()
        x = F.relu(x)
        print("relu2",x.shape)
        print()
        x = F.max_pool2d(x, (2,2))
        print("max_pool2d 2",x.shape)
        print()
        x = self.conv3(x)
        print("conv3", x.shape)
        print()
        x = F.relu(x)        
        print("relu3",x.shape)
        print()
        x = F.max_pool2d(x, (2,2))       
        print("max_pool2d 3",x.shape)        
        print()
        # print(x)
        
        if self._to_linear is None:
            print(x[0].shape[0])
            print(x[0].shape[1])
            print(x[0].shape[2])
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
        
        return x

        

    def forward(self, x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear)  # .view is reshape ... this flattens X before 
        x = F.relu(self.fc1(x))
        x = self.fc2(x) # bc this is our output layer. No activation here.
        return F.softmax(x, dim=1)


net = Net()
print(net)

initial torch.Size([1, 1, 100, 100])

conv1 torch.Size([1, 32, 96, 96])

relu1 torch.Size([1, 32, 96, 96])

max_pool2d 1 torch.Size([1, 32, 48, 48])

conv2 torch.Size([1, 64, 44, 44])

relu2 torch.Size([1, 64, 44, 44])

max_pool2d 2 torch.Size([1, 64, 22, 22])

conv3 torch.Size([1, 128, 18, 18])

relu3 torch.Size([1, 128, 18, 18])

max_pool2d 3 torch.Size([1, 128, 9, 9])

128
9
9
Net(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=10368, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=2, bias=True)
)


In [ ]:
import torch.optim as optim #設定優化器

optimizer = optim.Adam(net.parameters(), lr=0.001)
loss_function = nn.CrossEntropyLoss()

In [ ]:
X = torch.Tensor([i[0] for i in training_data]).view(-1,100,100) #np.array的張量
X = X/255.0 
y = torch.Tensor([i[1] for i in training_data]) #對角矩陣的張量

In [ ]:
VAL_PCT = 0.1  # lets reserve 10% of our data for validation
val_size = int(len(X)*VAL_PCT)
print(val_size)

2494


In [ ]:
train_X = X[:-val_size]
train_y = y[:-val_size]

test_X = X[-val_size:]
test_y = y[-val_size:]

In [ ]:
print(len(train_X), len(test_X))

22452 2494


In [ ]:
BATCH_SIZE = 100
EPOCHS = 1

for epoch in range(EPOCHS):
    for i in tqdm(range(0, len(train_X), BATCH_SIZE)): # from 0, to the len of x, stepping BATCH_SIZE at a time. [:50] ..for now just to dev
        #print(f"{i}:{i+BATCH_SIZE}")
        batch_X = train_X[i:i+BATCH_SIZE].view(-1, 1,100,100) #(-1,1)扁平化顯示
        batch_y = train_y[i:i+BATCH_SIZE]
        net.zero_grad() #歸零梯度,因為在pytorch中會累加梯度
        
        outputs = net(batch_X)
        loss = loss_function(outputs, batch_y)
        loss.backward()
        optimizer.step()    # Does the update

    print(f"Epoch: {epoch}. Loss: {loss}")
  


  0%|          | 0/225 [00:00<?, ?it/s]

initial torch.Size([100, 1, 100, 100])

conv1 torch.Size([100, 32, 96, 96])

relu1 torch.Size([100, 32, 96, 96])

max_pool2d 1 torch.Size([100, 32, 48, 48])

conv2 torch.Size([100, 64, 44, 44])

relu2 torch.Size([100, 64, 44, 44])

max_pool2d 2 torch.Size([100, 64, 22, 22])



  0%|          | 0/225 [00:01<?, ?it/s]

conv3 torch.Size([100, 128, 18, 18])

relu3 torch.Size([100, 128, 18, 18])

max_pool2d 3 torch.Size([100, 128, 9, 9])



RuntimeError: ignored

In [ ]:
correct = 0
total = 0
#當我們在做evaluating的時候（不需要計算導數），我們可以將推斷（inference）的代碼包裹在with torch.no_grad():之中，以達到暫時不追踪網絡參數中的導數的目的，總之是為了減少可能存在的計算和內存消耗
with torch.no_grad():
    for i in tqdm(range(len(test_X))):
        real_class = torch.argmax(test_y[i])#可能為貓{1,0}或狗{0,1},argmax找到座標,例如tensor(0)表示在貓那裏產生最大,tensor(1)表示在狗產生最大
  
        net_out = net(test_X[i].view(-1, 1, 100,100))[0]  # returns a list, 

        predicted_class = torch.argmax(net_out)

        if predicted_class == real_class:
            correct += 1
        total += 1
print("Accuracy: ", round(correct/total, 3))

  0%|          | 7/2494 [00:00<00:37, 66.35it/s]

initial torch.Size([1, 1, 100, 100])

conv1 torch.Size([1, 32, 96, 96])

relu1 torch.Size([1, 32, 96, 96])

max_pool2d 1 torch.Size([1, 32, 48, 48])

conv2 torch.Size([1, 64, 44, 44])

relu2 torch.Size([1, 64, 44, 44])

max_pool2d 2 torch.Size([1, 64, 22, 22])

conv3 torch.Size([1, 128, 18, 18])

relu3 torch.Size([1, 128, 18, 18])

max_pool2d 3 torch.Size([1, 128, 9, 9])

initial torch.Size([1, 1, 100, 100])

conv1 torch.Size([1, 32, 96, 96])

relu1 torch.Size([1, 32, 96, 96])

max_pool2d 1 torch.Size([1, 32, 48, 48])

conv2 torch.Size([1, 64, 44, 44])

relu2 torch.Size([1, 64, 44, 44])

max_pool2d 2 torch.Size([1, 64, 22, 22])

conv3 torch.Size([1, 128, 18, 18])

relu3 torch.Size([1, 128, 18, 18])

max_pool2d 3 torch.Size([1, 128, 9, 9])

initial torch.Size([1, 1, 100, 100])

conv1 torch.Size([1, 32, 96, 96])

relu1 torch.Size([1, 32, 96, 96])

max_pool2d 1 torch.Size([1, 32, 48, 48])

conv2 torch.Size([1, 64, 44, 44])

relu2 torch.Size([1, 64, 44, 44])

max_pool2d 2 torch.Size([1, 64

  1%|          | 21/2494 [00:00<00:38, 63.97it/s]

conv3 torch.Size([1, 128, 18, 18])

relu3 torch.Size([1, 128, 18, 18])

max_pool2d 3 torch.Size([1, 128, 9, 9])

initial torch.Size([1, 1, 100, 100])

conv1 torch.Size([1, 32, 96, 96])

relu1 torch.Size([1, 32, 96, 96])

max_pool2d 1 torch.Size([1, 32, 48, 48])

conv2 torch.Size([1, 64, 44, 44])

relu2 torch.Size([1, 64, 44, 44])

max_pool2d 2 torch.Size([1, 64, 22, 22])

conv3 torch.Size([1, 128, 18, 18])

relu3 torch.Size([1, 128, 18, 18])

max_pool2d 3 torch.Size([1, 128, 9, 9])

initial torch.Size([1, 1, 100, 100])

conv1 torch.Size([1, 32, 96, 96])

relu1 torch.Size([1, 32, 96, 96])

max_pool2d 1 torch.Size([1, 32, 48, 48])

conv2 torch.Size([1, 64, 44, 44])

relu2 torch.Size([1, 64, 44, 44])

max_pool2d 2 torch.Size([1, 64, 22, 22])

conv3 torch.Size([1, 128, 18, 18])

relu3 torch.Size([1, 128, 18, 18])

max_pool2d 3 torch.Size([1, 128, 9, 9])

initial torch.Size([1, 1, 100, 100])

conv1 torch.Size([1, 32, 96, 96])

relu1 torch.Size([1, 32, 96, 96])

max_pool2d 1 torch.Size([1, 3

  1%|▏         | 35/2494 [00:00<00:37, 64.75it/s]

...

relu2 torch.Size([1, 64, 44, 44])

max_pool2d 2 torch.Size([1, 64, 22, 22])

conv3 torch.Size([1, 128, 18, 18])

relu3 torch.Size([1, 128, 18, 18])

max_pool2d 3 torch.Size([1, 128, 9, 9])

initial torch.Size([1, 1, 100, 100])

conv1 torch.Size([1, 32, 96, 96])

relu1 torch.Size([1, 32, 96, 96])

max_pool2d 1 torch.Size([1, 32, 48, 48])

conv2 torch.Size([1, 64, 44, 44])

relu2 torch.Size([1, 64, 44, 44])

max_pool2d 2 torch.Size([1, 64, 22, 22])

conv3 torch.Size([1, 128, 18, 18])

relu3 torch.Size([1, 128, 18, 18])

max_pool2d 3 torch.Size([1, 128, 9, 9])

initial torch.Size([1, 1, 100, 100])

conv1 torch.Size([1, 32, 96, 96])

relu1 torch.Size([1, 32, 96, 96])

max_pool2d 1 torch.Size([1, 32, 48, 48])

conv2 torch.Size([1, 64, 44, 44])

relu2 torch.Size([1, 64, 44, 44])

max_pool2d 2 torch.Size([1, 64, 22, 22])

conv3 torch.Size([1, 128, 18, 18])

relu3 torch.Size([1, 128, 18, 18])

max_pool2d 3 torch.Size([1, 128, 9, 9])

initial torch.Size([1, 1, 100, 100])

conv1 torch.Size([1, 3

 99%|█████████▉| 2480/2494 [00:37<00:00, 67.17it/s]

torch.Size([1, 128, 18, 18])

max_pool2d 3 torch.Size([1, 128, 9, 9])

initial torch.Size([1, 1, 100, 100])

conv1 torch.Size([1, 32, 96, 96])

relu1 torch.Size([1, 32, 96, 96])

max_pool2d 1 torch.Size([1, 32, 48, 48])

conv2 torch.Size([1, 64, 44, 44])

relu2 torch.Size([1, 64, 44, 44])

max_pool2d 2 torch.Size([1, 64, 22, 22])

conv3 torch.Size([1, 128, 18, 18])

relu3 torch.Size([1, 128, 18, 18])

max_pool2d 3 torch.Size([1, 128, 9, 9])

initial torch.Size([1, 1, 100, 100])

conv1 torch.Size([1, 32, 96, 96])

relu1 torch.Size([1, 32, 96, 96])

max_pool2d 1 torch.Size([1, 32, 48, 48])

conv2 torch.Size([1, 64, 44, 44])

relu2 torch.Size([1, 64, 44, 44])

max_pool2d 2 torch.Size([1, 64, 22, 22])

conv3 torch.Size([1, 128, 18, 18])

relu3 torch.Size([1, 128, 18, 18])

max_pool2d 3 torch.Size([1, 128, 9, 9])

initial torch.Size([1, 1, 100, 100])

conv1 torch.Size([1, 32, 96, 96])

relu1 torch.Size([1, 32, 96, 96])

max_pool2d 1 torch.Size([1, 32, 48, 48])

conv2 torch.Size([1, 64, 44, 

100%|██████████| 2494/2494 [00:37<00:00, 66.41it/s]

max_pool2d 1 torch.Size([1, 32, 48, 48])

conv2 torch.Size([1, 64, 44, 44])

relu2 torch.Size([1, 64, 44, 44])

max_pool2d 2 torch.Size([1, 64, 22, 22])

conv3 torch.Size([1, 128, 18, 18])

relu3 torch.Size([1, 128, 18, 18])

max_pool2d 3 torch.Size([1, 128, 9, 9])

initial torch.Size([1, 1, 100, 100])

conv1 torch.Size([1, 32, 96, 96])

relu1 torch.Size([1, 32, 96, 96])

max_pool2d 1 torch.Size([1, 32, 48, 48])

conv2 torch.Size([1, 64, 44, 44])

relu2 torch.Size([1, 64, 44, 44])

max_pool2d 2 torch.Size([1, 64, 22, 22])

conv3 torch.Size([1, 128, 18, 18])

relu3 torch.Size([1, 128, 18, 18])

max_pool2d 3 torch.Size([1, 128, 9, 9])

initial torch.Size([1, 1, 100, 100])

conv1 torch.Size([1, 32, 96, 96])

relu1 torch.Size([1, 32, 96, 96])

max_pool2d 1 torch.Size([1, 32, 48, 48])

conv2 torch.Size([1, 64, 44, 44])

relu2 torch.Size([1, 64, 44, 44])

max_pool2d 2 torch.Size([1, 64, 22, 22])

conv3 torch.Size([1, 128, 18, 18])

relu3 torch.Size([1, 128, 18, 18])

max_pool2d 3 torch.Size(

In [ ]:
torch.cuda.is_available()

NameError: ignored

In [ ]:
device = torch.device("cuda:0")
device

NameError: ignored

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc. 
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

In [ ]:
torch.cuda.device_count()

In [ ]:
net.to(device)

Net(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=512, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=2, bias=True)
)

In [ ]:
net = Net().to(device)

initial torch.Size([1, 1, 50, 50])

conv1 torch.Size([1, 32, 46, 46])

relu1 torch.Size([1, 32, 46, 46])

max_pool2d 1 torch.Size([1, 32, 23, 23])

conv2 torch.Size([1, 64, 19, 19])

relu2 torch.Size([1, 64, 19, 19])

max_pool2d 2 torch.Size([1, 64, 9, 9])

conv3 torch.Size([1, 128, 5, 5])

relu3 torch.Size([1, 128, 5, 5])

max_pool2d 3 torch.Size([1, 128, 2, 2])

128
2
2


In [ ]:
EPOCHS = 3

def train(net):
    optimizer = optim.Adam(net.parameters(), lr=0.001)
    BATCH_SIZE = 100
    EPOCHS = 3
    for epoch in range(EPOCHS):
        for i in range(0, len(train_X), BATCH_SIZE): # from 0, to the len of x, stepping BATCH_SIZE at a time. [:50] ..for now just to dev
            #print(f"{i}:{i+BATCH_SIZE}")
            batch_X = train_X[i:i+BATCH_SIZE].view(-1, 1, 50, 50)
            batch_y = train_y[i:i+BATCH_SIZE]

            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            net.zero_grad()

            optimizer.zero_grad()   # zero the gradient buffers
            outputs = net(batch_X)
            loss = loss_function(outputs, batch_y)
            loss.backward()
            optimizer.step()    # Does the update

        print(f"Epoch: {epoch}. Loss: {loss}")

train(net)

initial torch.Size([100, 1, 50, 50])

conv1 torch.Size([100, 32, 46, 46])

relu1 torch.Size([100, 32, 46, 46])

max_pool2d 1 torch.Size([100, 32, 23, 23])

conv2 torch.Size([100, 64, 19, 19])

relu2 torch.Size([100, 64, 19, 19])

max_pool2d 2 torch.Size([100, 64, 9, 9])

conv3 torch.Size([100, 128, 5, 5])

relu3 torch.Size([100, 128, 5, 5])

max_pool2d 3 torch.Size([100, 128, 2, 2])



RuntimeError: ignored

In [ ]:
test_X.to(device)
test_y.to(device)

def test(net):
    correct = 0
    total = 0
    with torch.no_grad():
        for i in tqdm(range(len(test_X))):
            real_class = torch.argmax(test_y[i]).to(device)
            net_out = net(test_X[i].view(-1, 1, 50, 50).to(device))[0]  # returns a list, 
            predicted_class = torch.argmax(net_out)

            if predicted_class == real_class:
                correct += 1
            total += 1

    print("Accuracy: ", round(correct/total, 3))

test(net)

  1%|          | 25/2494 [00:00<00:10, 245.06it/s]

initial torch.Size([1, 1, 50, 50])

conv1 torch.Size([1, 32, 46, 46])

relu1 torch.Size([1, 32, 46, 46])

max_pool2d 1 torch.Size([1, 32, 23, 23])

conv2 torch.Size([1, 64, 19, 19])

relu2 torch.Size([1, 64, 19, 19])

max_pool2d 2 torch.Size([1, 64, 9, 9])

conv3 torch.Size([1, 128, 5, 5])

relu3 torch.Size([1, 128, 5, 5])

max_pool2d 3 torch.Size([1, 128, 2, 2])

initial torch.Size([1, 1, 50, 50])

conv1 torch.Size([1, 32, 46, 46])

relu1 torch.Size([1, 32, 46, 46])

max_pool2d 1 torch.Size([1, 32, 23, 23])

conv2 torch.Size([1, 64, 19, 19])

relu2 torch.Size([1, 64, 19, 19])

max_pool2d 2 torch.Size([1, 64, 9, 9])

conv3 torch.Size([1, 128, 5, 5])

relu3 torch.Size([1, 128, 5, 5])

max_pool2d 3 torch.Size([1, 128, 2, 2])

initial torch.Size([1, 1, 50, 50])

conv1 torch.Size([1, 32, 46, 46])

relu1 torch.Size([1, 32, 46, 46])

max_pool2d 1 torch.Size([1, 32, 23, 23])

conv2 torch.Size([1, 64, 19, 19])

relu2 torch.Size([1, 64, 19, 19])

max_pool2d 2 torch.Size([1, 64, 9, 9])

conv3 to

  3%|▎         | 85/2494 [00:00<00:09, 266.65it/s]

relu3 torch.Size([1, 128, 5, 5])

max_pool2d 3 torch.Size([1, 128, 2, 2])

initial torch.Size([1, 1, 50, 50])

conv1 torch.Size([1, 32, 46, 46])

relu1 torch.Size([1, 32, 46, 46])

max_pool2d 1 torch.Size([1, 32, 23, 23])

conv2 torch.Size([1, 64, 19, 19])

relu2 torch.Size([1, 64, 19, 19])

max_pool2d 2 torch.Size([1, 64, 9, 9])

conv3 torch.Size([1, 128, 5, 5])

relu3 torch.Size([1, 128, 5, 5])

max_pool2d 3 torch.Size([1, 128, 2, 2])

initial torch.Size([1, 1, 50, 50])

conv1 torch.Size([1, 32, 46, 46])

relu1 torch.Size([1, 32, 46, 46])

max_pool2d 1 torch.Size([1, 32, 23, 23])

conv2 torch.Size([1, 64, 19, 19])

relu2 torch.Size([1, 64, 19, 19])

max_pool2d 2 torch.Size([1, 64, 9, 9])

conv3 torch.Size([1, 128, 5, 5])

relu3 torch.Size([1, 128, 5, 5])

max_pool2d 3 torch.Size([1, 128, 2, 2])

initial torch.Size([1, 1, 50, 50])

conv1 torch.Size([1, 32, 46, 46])

relu1 torch.Size([1, 32, 46, 46])

max_pool2d 1 torch.Size([1, 32, 23, 23])

conv2 torch.Size([1, 64, 19, 19])

relu2 to

  6%|▌         | 142/2494 [00:00<00:08, 272.19it/s]

...

conv2 torch.Size([1, 64, 19, 19])

relu2 torch.Size([1, 64, 19, 19])

max_pool2d 2 torch.Size([1, 64, 9, 9])

conv3 torch.Size([1, 128, 5, 5])

relu3 torch.Size([1, 128, 5, 5])

max_pool2d 3 torch.Size([1, 128, 2, 2])

initial torch.Size([1, 1, 50, 50])

conv1 torch.Size([1, 32, 46, 46])

relu1 torch.Size([1, 32, 46, 46])

max_pool2d 1 torch.Size([1, 32, 23, 23])

conv2 torch.Size([1, 64, 19, 19])

relu2 torch.Size([1, 64, 19, 19])

max_pool2d 2 torch.Size([1, 64, 9, 9])

conv3 torch.Size([1, 128, 5, 5])

relu3 torch.Size([1, 128, 5, 5])

max_pool2d 3 torch.Size([1, 128, 2, 2])

initial torch.Size([1, 1, 50, 50])

conv1 torch.Size([1, 32, 46, 46])

relu1 torch.Size([1, 32, 46, 46])

max_pool2d 1 torch.Size([1, 32, 23, 23])

conv2 torch.Size([1, 64, 19, 19])

relu2 torch.Size([1, 64, 19, 19])

max_pool2d 2 torch.Size([1, 64, 9, 9])

conv3 torch.Size([1, 128, 5, 5])

relu3 torch.Size([1, 128, 5, 5])

max_pool2d 3 torch.Size([1, 128, 2, 2])

initial torch.Size([1, 1, 50, 50])

conv1 torch

 98%|█████████▊| 2455/2494 [00:08<00:00, 287.49it/s]

conv3 torch.Size([1, 128, 5, 5])

relu3 torch.Size([1, 128, 5, 5])

max_pool2d 3 torch.Size([1, 128, 2, 2])

initial torch.Size([1, 1, 50, 50])

conv1 torch.Size([1, 32, 46, 46])

relu1 torch.Size([1, 32, 46, 46])

max_pool2d 1 torch.Size([1, 32, 23, 23])

conv2 torch.Size([1, 64, 19, 19])

relu2 torch.Size([1, 64, 19, 19])

max_pool2d 2 torch.Size([1, 64, 9, 9])

conv3 torch.Size([1, 128, 5, 5])

relu3 torch.Size([1, 128, 5, 5])

max_pool2d 3 torch.Size([1, 128, 2, 2])

initial torch.Size([1, 1, 50, 50])

conv1 torch.Size([1, 32, 46, 46])

relu1 torch.Size([1, 32, 46, 46])

max_pool2d 1 torch.Size([1, 32, 23, 23])

conv2 torch.Size([1, 64, 19, 19])

relu2 torch.Size([1, 64, 19, 19])

max_pool2d 2 torch.Size([1, 64, 9, 9])

conv3 torch.Size([1, 128, 5, 5])

relu3 torch.Size([1, 128, 5, 5])

max_pool2d 3 torch.Size([1, 128, 2, 2])

initial torch.Size([1, 1, 50, 50])

conv1 torch.Size([1, 32, 46, 46])

relu1 torch.Size([1, 32, 46, 46])

max_pool2d 1 torch.Size([1, 32, 23, 23])

conv2 tor

100%|██████████| 2494/2494 [00:08<00:00, 289.68it/s]

max_pool2d 1 torch.Size([1, 32, 23, 23])

conv2 torch.Size([1, 64, 19, 19])

relu2 torch.Size([1, 64, 19, 19])

max_pool2d 2 torch.Size([1, 64, 9, 9])

conv3 torch.Size([1, 128, 5, 5])

relu3 torch.Size([1, 128, 5, 5])

max_pool2d 3 torch.Size([1, 128, 2, 2])

initial torch.Size([1, 1, 50, 50])

conv1 torch.Size([1, 32, 46, 46])

relu1 torch.Size([1, 32, 46, 46])

max_pool2d 1 torch.Size([1, 32, 23, 23])

conv2 torch.Size([1, 64, 19, 19])

relu2 torch.Size([1, 64, 19, 19])

max_pool2d 2 torch.Size([1, 64, 9, 9])

conv3 torch.Size([1, 128, 5, 5])

relu3 torch.Size([1, 128, 5, 5])

max_pool2d 3 torch.Size([1, 128, 2, 2])

initial torch.Size([1, 1, 50, 50])

conv1 torch.Size([1, 32, 46, 46])

relu1 torch.Size([1, 32, 46, 46])

max_pool2d 1 torch.Size([1, 32, 23, 23])

conv2 torch.Size([1, 64, 19, 19])

relu2 torch.Size([1, 64, 19, 19])

max_pool2d 2 torch.Size([1, 64, 9, 9])

conv3 torch.Size([1, 128, 5, 5])

relu3 torch.Size([1, 128, 5, 5])

max_pool2d 3 torch.Size([1, 128, 2, 2])

initi

In [ ]:
correct = 0
total = 0
for i in tqdm(range(0, len(test_X), BATCH_SIZE)):

    batch_X = test_X[i:i+BATCH_SIZE].view(-1, 1, 50, 50).to(device)
    batch_y = test_y[i:i+BATCH_SIZE].to(device)
    batch_out = net(batch_X)

    out_maxes = [torch.argmax(i) for i in batch_out]
    target_maxes = [torch.argmax(i) for i in batch_y]
    for i,j in zip(out_maxes, target_maxes):
        if i == j:
            correct += 1
        total += 1
print("Accuracy: ", round(correct/total, 3))

 36%|███▌      | 9/25 [00:00<00:00, 81.92it/s]

initial torch.Size([100, 1, 50, 50])

conv1 torch.Size([100, 32, 46, 46])

relu1 torch.Size([100, 32, 46, 46])

max_pool2d 1 torch.Size([100, 32, 23, 23])

conv2 torch.Size([100, 64, 19, 19])

relu2 torch.Size([100, 64, 19, 19])

max_pool2d 2 torch.Size([100, 64, 9, 9])

conv3 torch.Size([100, 128, 5, 5])

relu3 torch.Size([100, 128, 5, 5])

max_pool2d 3 torch.Size([100, 128, 2, 2])

initial torch.Size([100, 1, 50, 50])

conv1 torch.Size([100, 32, 46, 46])

relu1 torch.Size([100, 32, 46, 46])

max_pool2d 1 torch.Size([100, 32, 23, 23])

conv2 torch.Size([100, 64, 19, 19])

relu2 torch.Size([100, 64, 19, 19])

max_pool2d 2 torch.Size([100, 64, 9, 9])

conv3 torch.Size([100, 128, 5, 5])

relu3 torch.Size([100, 128, 5, 5])

max_pool2d 3 torch.Size([100, 128, 2, 2])

initial torch.Size([100, 1, 50, 50])

conv1 torch.Size([100, 32, 46, 46])

relu1 torch.Size([100, 32, 46, 46])

max_pool2d 1 torch.Size([100, 32, 23, 23])

conv2 torch.Size([100, 64, 19, 19])

relu2 torch.Size([100, 64, 19, 19

100%|██████████| 25/25 [00:00<00:00, 81.15it/s]

max_pool2d 2 torch.Size([100, 64, 9, 9])

conv3 torch.Size([100, 128, 5, 5])

relu3 torch.Size([100, 128, 5, 5])

max_pool2d 3 torch.Size([100, 128, 2, 2])

initial torch.Size([100, 1, 50, 50])

conv1 torch.Size([100, 32, 46, 46])

relu1 torch.Size([100, 32, 46, 46])

max_pool2d 1 torch.Size([100, 32, 23, 23])

conv2 torch.Size([100, 64, 19, 19])

relu2 torch.Size([100, 64, 19, 19])

max_pool2d 2 torch.Size([100, 64, 9, 9])

conv3 torch.Size([100, 128, 5, 5])

relu3 torch.Size([100, 128, 5, 5])

max_pool2d 3 torch.Size([100, 128, 2, 2])

initial torch.Size([100, 1, 50, 50])

conv1 torch.Size([100, 32, 46, 46])

relu1 torch.Size([100, 32, 46, 46])

max_pool2d 1 torch.Size([100, 32, 23, 23])

conv2 torch.Size([100, 64, 19, 19])

relu2 torch.Size([100, 64, 19, 19])

max_pool2d 2 torch.Size([100, 64, 9, 9])

conv3 torch.Size([100, 128, 5, 5])

relu3 torch.Size([100, 128, 5, 5])

max_pool2d 3 torch.Size([100, 128, 2, 2])

initial torch.Size([100, 1, 50, 50])

conv1 torch.Size([100, 32, 46, 